<a href="https://colab.research.google.com/github/VinKKAP/Data-Analysis-with-LLM/blob/main/TryTask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

# Upgrade pip und installiere die benötigten Pakete
%pip install --upgrade pip
%pip install --upgrade torch transformers
import torch

#from google.colab import userdata
#userdata.get('HF_TOKEN')

# Überprüfen, ob CUDA verfügbar ist, und den Namen des CUDA-Geräts abrufen
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
device

device(type='cuda')

GPU ist verbunden für die berechnung, aber die

1.   Listeneintrag
2.   Listeneintrag

Instruction für das Modell scheitert daran das dass jeweilige Modell nicht richtig im Code aus dem Buch funktioniert.

In [4]:
import sqlite3

In [5]:
conn = sqlite3.connect(':memory:')

version = sqlite3.version
print(version)


2.6.0


In [6]:

import argparse
import torch
import re
import time
from transformers import pipeline

def create_prompt(question):  #1
    """ Generate prompt to translate question into SQL query.

    Args:
        question: question about data in natural language.

    Returns:
        prompt for question translation.
    """
    parts = []
    parts += ['Database:']
    parts += ['create table games(rank int, name text, platform text,']
    parts += ['year int, genre text, publisher text, americasales numeric,']
    parts += ['eusales numeric, japansales numeric, othersales numeric,']
    parts += ['globalsales numeric);']
    parts += ['Translate this question into SQL query:']
    parts += [question]
    return '\n'.join(parts)

def call_llm(prompt):  #2
    """ Query large language model and return answer.

    Args:
        prompt: input prompt for language model.

    Returns:
        Answer by language model.
    """
    pipe = pipeline("text-generation", model="Qwen/Qwen2-1.5B-Instruct", max_length=600, device=device, truncation=True)
    for nr_retries in range(1, 4):
        try:
            response = pipe(prompt, max_length=100)
            return response[0]['generated_text']
        except Exception as e:
            print(f"Error: {e}")
            time.sleep(nr_retries * 2)
    raise Exception('Cannot query model!')

if __name__ == '__main__':  #3
    parser = argparse.ArgumentParser()
    parser.add_argument('question', type=str, help='A question about games')
    args = parser.parse_args(['What are the most sold games?'])

    prompt = create_prompt(args.question)
    answer = call_llm(prompt) #4
    query = re.findall(r'SELECT.*?;', answer, re.DOTALL)
    if query:
        print(f'SQL: {query[0]}')
    else:
        print("No SQL query found in the response.")

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

SQL: SELECT name FROM games ORDER BY americasales DESC LIMIT 10;


In [7]:
print(prompt)

Database:
create table games(rank int, name text, platform text,
year int, genre text, publisher text, americasales numeric,
eusales numeric, japansales numeric, othersales numeric,
globalsales numeric);
Translate this question into SQL query:
What are the most sold games?


In [8]:
print(answer)

Database:
create table games(rank int, name text, platform text,
year int, genre text, publisher text, americasales numeric,
eusales numeric, japansales numeric, othersales numeric,
globalsales numeric);
Translate this question into SQL query:
What are the most sold games? SELECT name FROM games ORDER BY americasales DESC LIMIT 10;
Here is the SQL code to get the most sold games. It orders all the games by their Americas sales in descending order and then


In [9]:
from transformers import pipeline

messages = [

            {"role": "user", "content": prompt},

           ]

pipe = pipeline("text-generation", model="Qwen/Qwen2-1.5B-Instruct", max_length=900, device='cuda')

pipe(messages)

[{'generated_text': [{'role': 'user',
    'content': 'Database:\ncreate table games(rank int, name text, platform text,\nyear int, genre text, publisher text, americasales numeric,\neusales numeric, japansales numeric, othersales numeric,\nglobalsales numeric);\nTranslate this question into SQL query:\nWhat are the most sold games?'},
   {'role': 'assistant',
    'content': 'To find the most sold games, you can use the following SQL query:\n\n```\nSELECT name, globalsales \nFROM games \nORDER BY globalsales DESC \nLIMIT 1;\n```\n\nThis query selects the `name` and `globalsales` columns from the `games` table, orders them in descending order by `globalsales`, and then limits the result to only one row, which represents the game with the highest sales.'}]}]

In [ ]:
conn = sqlite3.connect('games.db')


In [ ]:
import argparse
import torch
import re
import time
from transformers import pipeline
import sqlite3
import sys

# Connect to the database
#conn = sqlite3.connect('games.db')
#cursor = conn.cursor()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def get_structure(data_path): #1 Extracts the database structure
    """ Extract structure from SQLite database.

    Args:
        data_path: path to SQLite data file.

    Returns:
        text description of database structure.
    """
    with sqlite3.connect(data_path) as connection:
        cursor = connection.cursor()
        cursor.execute("select sql from sqlite_master where type = 'table';")
        table_rows = cursor.fetchall()
        table_ddls = [r[0] for r in table_rows]
    return '\n'.join(table_ddls)

def create_prompt(description, question): #2 Creates a prompt for translation
    """ Generate prompt to translate a question into an SQL query.

    Args:
        description: text description of database structure.
        question: question about data in natural language.

    Returns:
        prompt for question translation.
    """
    parts = []
    parts += ['Database:']
    parts += [description]
    parts += ['Translate this question into SQL query:']
    parts += [question]
    return '\n'.join(parts)

def call_llm(prompt): #3 Invokes the language model
    """ Query large language model and return answer.

    Args:
        prompt: input prompt for language model.

    Returns:
        Answer by language model.
    """
    pipe = pipeline("text-generation", model="Qwen/Qwen2-1.5B-Instruct", max_length=800, device=device, truncation=True, temperature=0.7)
    
    for nr_retries in range(1, 4):
        try:
            response = pipe(prompt, max_length=800)
            return response[0]['generated_text']
        except:
            time.sleep(nr_retries * 2)
    raise Exception('Cannot query model!')

def process_query(data_path, query): #4 Processes a query on a database
    """ Processes SQL query and returns result.

    Args:
        data_path: path to SQLite data file.
        query: process this query on database.

    Returns:
        query result.
    """
    with sqlite3.connect(data_path) as connection:
        cursor = connection.cursor()
        cursor.execute(query)
        table_rows = cursor.fetchall()
        table_strings = [str(r) for r in table_rows]
        return '\n'.join(table_strings)

if __name__ == '__main__':
    # Simulate command-line arguments
    #sys.argv = ['ipykernel_launcher.py', r'C:\Users\kappv\Desktop\Data-Analysis-with-LLM-3\games.db']
    parser = argparse.ArgumentParser()
    parser.add_argument('dbpath', type=str, help='Path to SQLite data')
    args = parser.parse_args()
    
    data_structure = get_structure(args.dbpath) #5 Reads data structure
    while True: #6  Answers questions until the user quits
        user_input = input('Enter question:')
        if user_input == 'quit':
            break
        prompt = create_prompt(data_structure, user_input)
        answer = call_llm(prompt)
        query = re.findall('```sql(.*)```', answer, re.DOTALL)[0]
        print(f'SQL: {query}')

        try: #7  Processes query on the database
            result = process_query(args.data_path, query)
            print(f'Result: {result}')
        except:
            print('Error processing query! Try to reformulate.')


SQL: 
SELECT COUNT(*) FROM games WHERE year = 2017;
```

This query selects all rows from the `games` table where the `year` column is equal to 2017. The `COUNT(*)` function then counts the number of matches found and returns that count as the result. If you want to display the results only for the first game in 2017 (assuming there's at least one), you could modify the query like so:

```sql
SELECT * FROM games WHERE year = 2017 LIMIT 1;
``` 

This will return the first row of data in the `games` table where the `year` column equals 2017. If there are no such records, it will return nothing. 

Remember to replace `games`, `rank`, `name`, etc., with your actual table and column names if they're different. Also, adjust the logic according to whether you need to include the global sales or not. If you only care about Americasales, EUSales, JapanSales, Othersales, and GlobalSales, you might want to select those instead of just `globalsales`. Similarly, if you only care about a specific ga

In [ ]:
print("LLM Response:", answer)


LLM Response: Database:
CREATE TABLE games(
    rank int, 
    name text, 
    platform text,
    year int, 
    genre text, 
    publisher text, 
    americasales numeric,
    eusales numeric, 
    japansales numeric, 
    othersales numeric,
    globalsales numeric
)
Translate this question into SQL query:
How many games in japan have been sold more than 5000 copies? SELECT COUNT(*) FROM games WHERE country = 'Japan' AND sales > 5000; This is a straightforward SQL query that counts the number of games in Japan with sales greater than 5000. The table has columns for 'rank', 'name', 'platform', 'year', 'genre', 'publisher', 'americasales', 'eusales', 'japansales', 'othersales', and 'globalsales'. The query selects all rows where the 'country' column is equal to 'Japan' and the'sales' column is greater than 5000, then returns the count of these rows as the result. The answer will be the number of games meeting both criteria. 

Please note: This assumes the data is stored in a database n

In [8]:
query = re.findall(r'```sql\s*(.*?)\s*```', answer, re.DOTALL)
if query:
    query = query[0]
else:
    raise ValueError("No SQL query found in the response.")


ValueError: No SQL query found in the response.